In [1]:
import logging
from importlib import reload
import shutil
import os

from marmopose.version import __version__ as marmopose_version
from marmopose.config import Config
from marmopose.processing.prediction import Predictor
from marmopose.visualization.display_2d import Visualizer2D
from marmopose.visualization.display_3d import Visualizer3D
from marmopose.processing.triangulation import Reconstructor3D

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(name)s - %(message)s')
logger = logging.getLogger(__name__)

logger.info(f'MarmoPose version: {marmopose_version}')

2026-01-09 15:22:29,738 - INFO - __main__ - MarmoPose version: 1.2.2


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
config_path = '../configs/default.yaml'

config = Config(
    config_path=config_path,
    
    n_tracks=1,
    project='../demos/single',
)
print(config.sub_directory)

2026-01-09 15:22:29,761 - INFO - marmopose.config - *** Overriding config *** | n_tracks: 2 -> 1
2026-01-09 15:22:29,763 - INFO - marmopose.config - *** Overriding config *** | project: ../demos/single -> ../demos/single


{'calibration': '..\\demos\\single\\calibration', 'points_2d': '..\\demos\\single\\points_2d', 'points_3d': '..\\demos\\single\\points_3d', 'videos_raw': '..\\demos\\single\\videos_raw', 'videos_labeled_2d': '..\\demos\\single\\videos_labeled_2d', 'videos_labeled_3d': '..\\demos\\single\\videos_labeled_3d'}


In [4]:
DIR = "C:\\Users\\rcagnol\\Documents\\ProjectCameras\\MarmoPose_dev\\Videos\\Test3.10"

destination_dir = 'path/to/destination/directory/'

for i in range(4):
    shutil.copy(DIR +f'/input/output{i+1}.mp4', config.sub_directory['videos_raw'])

os.makedirs(DIR + '/Output', exist_ok=True)
shutil.copytree(config.sub_directory['calibration'], DIR + '/Calib', dirs_exist_ok=True)


'C:\\Users\\rcagnol\\Documents\\ProjectCameras\\MarmoPose_dev\\Videos\\Test3.10/Calib'

In [5]:
import marmopose.processing.prediction as prediction
reload(prediction)
from marmopose.processing.prediction import Predictor
predictor = Predictor(config, batch_size=4)
predictor.predict()
# shutil.copytree(config.sub_directory['points_2d'], DIR + '/Output/points_2d', dirs_exist_ok=True)

2026-01-09 14:23:47,013 - INFO - marmopose.processing.prediction - Predictor device: cuda | batch size: 4
2026-01-09 14:23:47,017 - INFO - marmopose.processing.prediction - Load detection model config from ..\models\detection_model\config.py
2026-01-09 14:23:47,017 - INFO - marmopose.processing.prediction - Load detection model checkpoint from ..\models\detection_model\best.pth


Loads checkpoint by local backend from path: ..\models\detection_model\best.pth


2026-01-09 14:23:48,560 - INFO - marmopose.processing.prediction - Load pose model config from ..\models\pose_model\config.py
2026-01-09 14:23:48,561 - INFO - marmopose.processing.prediction - Load pose model checkpoint from ..\models\pose_model\best.pth


Loads checkpoint by local backend from path: ..\models\pose_model\best.pth


2026-01-09 14:23:49,063 - INFO - marmopose.processing.prediction - Predicting 2D poses for: ..\demos\single\videos_raw\output1.mp4
Predicting ... :   0%|                                                 | 0/4721 [00:00<?, ?frames/s]c:\Users\rcagnol\miniconda3\envs\marmopose_dev\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Predicting ... : 100%|██████████████████████████████████████| 4721/4721 [06:18<00:00, 12.47frames/s]
2026-01-09 14:30:07,962 - INFO - marmopose.utils.data_io - Overwriting existing output1_points in ..\demos\single\points_2d\original.h5
2026-01-09 14:30:07,963 - INFO - marmopose.utils.data_io - Overwriting existing output1_points in ..\demos\single\points_2d\original.h5
2026-01-09 14:30:07,969 - INF

In [11]:
from importlib import reload
import marmopose.processing.triangulation as triangulation
reload(triangulation)
import marmopose.processing.optimization as optimization
reload(optimization)
import marmopose.calibration.cameras as cameras
reload(cameras)
import marmopose.utils.data_io as data_io
reload(data_io)
from marmopose.processing.triangulation import Reconstructor3D

reconstructor_3d = Reconstructor3D(config)
reconstructor_3d.triangulate(file_names = predictor.file_names)
# shutil.copytree(config.sub_directory['points_3d'], DIR + '/Output/points_3d', dirs_exist_ok=True)

2026-01-09 15:10:26,898 - INFO - marmopose.processing.triangulation - Loaded camera group from: ..\demos\single\calibration\camera_params.json
2026-01-09 15:10:26,899 - INFO - marmopose.processing.triangulation - Loaded DAE from: ..\models\dae_model\best.pth
2026-01-09 15:10:26,954 - INFO - marmopose.processing.autoencoder - DAE device: cuda
2026-01-09 15:10:26,958 - INFO - marmopose.processing.triangulation - Enabled filling of missing values using Denoising Autoencoder
2026-01-09 15:10:26,974 - INFO - marmopose.utils.data_io - Loaded 2D points and bboxes from ..\demos\single\points_2d\original.h5 with order: ['bak-1-2', 'bak-2-2', 'bak-3-2', 'bak-4-2', 'output1', 'output2', 'output3', 'output4', 'output_video1', 'output_video2', 'output_video3', 'output_video4']
Triangulating... :  11%|███▉                                | 524/4721 [00:02<00:20, 201.36frames/s]


KeyboardInterrupt: 

In [11]:
import marmopose.visualization.display_2d as display_2d
reload(display_2d)
from marmopose.visualization.display_2d import Visualizer2D
visualizer_2d = Visualizer2D(config, with_gaze=True)
# visualizer_2d.generate_videos_2d(file_names = predictor.file_names)
visualizer_2d.generate_videos_2d(file_names = [f'output{i}' for i in range(1,5)])
# shutil.copytree(config.sub_directory['videos_labeled_2d'], DIR + '/Output/videos_labeled_2d', dirs_exist_ok=True)

2026-01-09 16:04:10,525 - INFO - marmopose.utils.data_io - Loaded 2D points and bboxes from ..\demos\single\points_2d\original.h5 with order: ['bak-1-2', 'bak-2-2', 'bak-3-2', 'bak-4-2', 'output1', 'output2', 'output3', 'output4', 'output_video1', 'output_video2', 'output_video3', 'output_video4']
2026-01-09 16:04:10,528 - INFO - marmopose.utils.data_io - Loaded 3D points from ..\demos\single\points_3d\optimized.h5 with order: ['track1']
2D Visualizing output1:   0%|                                          | 0/4721 [00:00<?, ?frames/s]


2D Visualizing output1:   0%|                                  | 1/4721 [00:00<10:17,  7.65frames/s]


2D Visualizing output1:   0%|                                  | 5/4721 [00:00<03:31, 22.28frames/s]


2D Visualizing output1:   0%|                                  | 9/4721 [00:00<02:57, 26.51frames/s]





2D Visualizing output1:   0%|                                 | 13/4721 [00:00<02:48, 27.98frames/s]


2D Visualizing output1:   0%|           

In [12]:
import marmopose.visualization.display_3d as display_3d
reload(display_3d)
from marmopose.visualization.display_3d import Visualizer3D
import marmopose.utils.data_io as data_io
reload(data_io)
visualizer_3d = Visualizer3D(config,with_gaze=True)
visualizer_3d.generate_video_3d(source_3d='optimized', video_type='composite', file_names_2d = [f'output{i}' for i in range(1,5)])
# shutil.copytree(config.sub_directory['videos_labeled_3d'], DIR + '/Output/videos_labeled_3d', dirs_exist_ok=True)

2026-01-09 16:07:33,514 - INFO - marmopose.utils.data_io - Loaded 3D points from ..\demos\single\points_3d\optimized.h5 with order: ['track1']
2026-01-09 16:07:34,620 - INFO - marmopose.visualization.display_3d - Generating composite video from frame 0 to 4721
Visualizing composite... : 100%|████████████████████████████| 4721/4721 [04:55<00:00, 15.98frames/s]


In [2]:
from marmopose.utils.data_io import load_points_3d_h5
points_3d = load_points_3d_h5("..\demos\single\points_3d\original.h5")
print(points_3d[:,0:10,0,:])
points_3d = load_points_3d_h5("..\demos\single\points_3d\optimized.h5")
print(points_3d[:,0:10,0,:])


2026-01-09 10:42:27,659 - INFO - marmopose.utils.data_io - Loaded 3D points from ..\demos\single\points_3d\original.h5 with order: ['track1']
2026-01-09 10:42:27,663 - INFO - marmopose.utils.data_io - Loaded 3D points from ..\demos\single\points_3d\optimized.h5 with order: ['track1']


[[[532.61051688 664.24564925 364.58982983]
  [521.05699173 665.58754447 378.40204483]
  [526.50235023 666.6744882  372.68898957]
  [520.03913553 666.7080353  372.61732692]
  [502.7833827  665.18774751 391.80123176]
  [495.77724862 665.39962321 396.10807362]
  [490.28215561 667.3104349  397.01787551]
  [492.12460317 677.59173044 389.96528671]
  [491.19997646 678.76190464 390.4743403 ]
  [490.73043171 679.3156089  390.32635208]]]
[[[524.08752566 659.83111176 374.8712232 ]
  [519.49863414 660.37055352 378.50228918]
  [514.36086315 660.79479694 382.05822272]
  [509.11375031 660.72911465 385.03758397]
  [504.06305128 660.26606727 388.31911953]
  [498.01627582 660.81928967 391.60088633]
  [491.32395492 663.25230878 392.69693132]
  [487.84037633 667.79975119 392.62543899]
  [485.76515333 671.85736039 389.72064132]
  [485.0659128  674.7985948  387.81404054]]]


In [65]:
from marmopose.calibration.cameras import CameraGroup
from marmopose.utils.data_io import load_points_bboxes_2d_h5
import numpy as np
from itertools import combinations
from typing import Any, Dict, List, Tuple

def triangulate_SVD(points: List[Tuple[float, float]], camera_mats: List[np.ndarray]) -> np.ndarray:
    num_cams = len(camera_mats)
    A = np.zeros((num_cams * 2, 4))
    for i in range(num_cams):
        x, y = points[i]
        mat = camera_mats[i]
        A[(i * 2):(i * 2 + 1)] = x * mat[2] - mat[0]
        A[(i * 2 + 1):(i * 2 + 2)] = y * mat[2] - mat[1]
    _, _, vh = np.linalg.svd(A, full_matrices=True)
    print(A)
    print(vh)
    p3d = vh[-1]
    p3d = p3d[:3] / p3d[3]
    return p3d


camera_group = CameraGroup.load_from_json("..\demos\single\calibration\camera_params.json")
points_2d = load_points_bboxes_2d_h5("..\demos\single\points_2d\original.h5", [f"output{i+1}" for i in range(4)])[0]
points_with_score_2d_flat = np.copy(points_2d[:,:,0,:,:]).reshape(points_2d.shape[0], points_2d.shape[1]*points_2d.shape[3], points_2d.shape[4])
# print(points_with_score_2d_flat[:,0,:])
# points_3d_flat = camera_group.triangulate_ransac(points_with_score_2d_flat, undistort=True)
# print(points_3d_flat[0,:])
cam_mats = np.array([cam.get_extrinsic_matrix() for cam in camera_group.cameras])
# n_points = points_2d_flat_undistorted.shape[1]
# points_3d_flat = np.full((n_points, 3), np.nan)
sub_scores = points_with_score_2d_flat[:,0,2]
sub_points = points_with_score_2d_flat[:,0,:2]
valid_points = ~np.isnan(sub_points[:, 0])
valid_indices = np.where(valid_points)[0]
print(sub_scores)
print(sub_points)
# for pt_idx in range(n_points):
# sub_points_distorted = points_2d_flat[:, pt_idx, :]
# sub_points = points_2d_flat_undistorted[:, pt_idx, :]
# sub_scores = scores_2d[:, pt_idx]
# valid_points = ~np.isnan(sub_points[:, 0])
# valid_indices = np.where(valid_points)[0]

for r in range(len(valid_indices)-1, len(valid_indices) + 1):
    for indices in combinations(valid_indices, r):
        print(indices)
        selected_points = sub_points[list(indices), :]
        selected_cam_mats = cam_mats[list(indices), :, :]

        points_3d = triangulate_SVD(selected_points, selected_cam_mats)

        print(points_3d)

        points_2d_reprojected = camera_group.reproject(points_3d[np.newaxis, :])
        reproj_errors = np.linalg.norm(points_2d_reprojected[valid_indices, 0, :] - points_with_score_2d_flat[valid_indices,0, :2], axis=1)

        mean_error = np.median(reproj_errors)

        print(mean_error)
# points_3d_flat[pt_idx] = best_point_3d


# print(cam_mats)


2025-12-15 16:47:32,481 - INFO - marmopose.utils.data_io - Loaded 2D points and bboxes from ..\demos\single\points_2d\original.h5 with order: ['bak-1-2', 'bak-2-2', 'bak-3-2', 'bak-4-2', 'output1', 'output2', 'output3', 'output4', 'output_video1', 'output_video2', 'output_video3', 'output_video4']


[0.69409597 0.7007305         nan 0.93876326]
[[ 636.95288086  910.66955566]
 [1175.40625     296.84429932]
 [          nan           nan]
 [ 717.52282715   81.475914  ]]
(0, 1)
[[ 3.50986200e+02  4.83438300e+02  2.20920957e+02 -5.46507567e+06]
 [ 5.00511977e+02  6.91640644e+02  3.16924313e+02 -7.79493016e+06]
 [ 8.01520968e+02  3.71275648e+02  7.75434965e+02 -1.07981586e+07]
 [ 2.02635351e+02  9.27859329e+01  1.96079635e+02 -2.72855655e+06]]
[[-7.00016204e-05 -5.72758042e-05 -5.86288109e-05  9.99999994e-01]
 [ 1.57919401e-01 -8.55874165e-01  4.92484393e-01 -9.09249285e-06]
 [-2.21100686e-02 -5.01682380e-01 -8.64769292e-01 -8.09823981e-05]
 [-9.87204438e-01 -1.25674982e-01  9.81488222e-02 -7.05496977e-05]]
[13993.03569954  1781.3681129  -1391.20117409]
7.419667728124286e+16
(0, 3)
[[ 3.50986200e+02  4.83438300e+02  2.20920957e+02 -5.46507567e+06]
 [ 5.00511977e+02  6.91640644e+02  3.16924313e+02 -7.79493016e+06]
 [ 5.90887143e+02  3.53421502e+02  2.01954533e+02 -8.50193861e+06]
 [ 6.76

In [74]:
from marmopose.utils.data_io import load_axes
from marmopose.utils.helpers import orthogonalize_vector
from pathlib import Path
def construct_transformation_matrix(camera_group, axes):
    offset = np.array(axes['offset'])
    cam_names = [key for key in axes.keys() if key != 'offset']
    axes_2d = np.array([axes[cam_name] for cam_name in cam_names], dtype=np.float32)
    print(axes_2d)
    
    sub_camera_group = camera_group.subset_cameras_names(cam_names)
    axes_3d = sub_camera_group.triangulate(axes_2d, undistort=True) - offset
    print(axes_3d)

    new_x_axis = axes_3d[1] - axes_3d[0]
    new_y_axis = orthogonalize_vector(axes_3d[2] - axes_3d[0], new_x_axis)
    new_z_axis = np.cross(new_x_axis, new_y_axis)
    
    R = np.vstack([new_x_axis, new_y_axis, new_z_axis])
    R /= np.linalg.norm(R, axis=1)[:, None]
    
    # Construct transformation matrix
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = -R @ axes_3d[0]

    return T

axes_path = Path("../demos/single/calibration/axes.json")
if not axes_path.exists():
    logger.info(f'Axes file not found in: {axes_path}')
else:
    logger.info('Updating extrinsics by user-defined axes')
    axes = load_axes(axes_path)
    T = construct_transformation_matrix(camera_group, axes)

2025-12-15 17:40:49,533 - INFO - __main__ - Updating extrinsics by user-defined axes


[[[1390.   81.]
  [1554.  873.]
  [ 653.  290.]]

 [[1084.  433.]
  [1411.  976.]
  [ 353.  595.]]]
[[ 1.39974973e+04  1.74845073e+03 -3.90626994e+02]
 [ 1.44684495e+04  1.31137664e+03 -9.68787768e-01]
 [ 1.45845774e+04  2.24880092e+03 -1.06315310e+03]]
